In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
city_data = pd.read_csv('../WeatherPy/city_data.csv')
city_data = city_data[['City', 'Lng', 'Lat', 'Temp_Max', 'Humidity',
       'Cloudiness', 'Wind_Speed', 'Country', 'Date']]
city_data

,City,Lng,Lat,Temp_Max,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Mar del Plata,-57.5575,-38.0023,292.15,59,40,6.69,AR,1619288449
1,Kimbe,150.1429,-5.5502,297.05,93,65,0.62,PG,1619288229
2,Kruisfontein,24.7314,-34.0033,288.15,92,90,0.89,ZA,1619288395
3,Busselton,115.3333,-33.6500,284.26,88,0,0.83,AU,1619288149
4,Bredasdorp,20.0403,-34.5322,285.15,87,96,2.57,ZA,1619288396
...,...,...,...,...,...,...,...,...,...
545,Tam Ky,108.4833,15.5667,300.15,94,40,1.03,VN,1619288929
546,Yar-Sale,70.8333,66.8333,270.39,93,94,1.66,RU,1619288655
547,Tres Arroyos,-60.2798,-38.3739,290.37,47,15,7.67,AR,1619288930
548,Evensk,159.2333,61.9500,265.37,80,100,4.83,RU,1619288656


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
m = gmaps.figure()
m.add_layer(gmaps.heatmap_layer(locations=city_data[['Lat','Lng']], weights=city_data['Humidity']))
m

Figure(layout=FigureLayout(height='420px'))

In [61]:
filtered_city_data = city_data.dropna()
filtered_city_data = filtered_city_data[(city_data['Temp_Max'] < 300) & 
                                        (city_data['Temp_Max'] > 295) & 
                                        (city_data['Wind_Speed'] < 10) &
                                        (city_data['Cloudiness'] < 20)]
filtered_city_data = filtered_city_data.reset_index()
del filtered_city_data['index']
filtered_city_data

,City,Lng,Lat,Temp_Max,Humidity,Cloudiness,Wind_Speed,Country,Date
0,San Vicente de Cañete,-76.3835,-13.0751,295.80,71,8,2.91,PE,1619288739
1,Castro,-50.0119,-24.7911,296.35,49,7,2.71,BR,1619288430
2,Kupang,123.5833,-10.1667,297.60,86,6,3.72,ID,1619288443
3,Patiāla,76.4003,30.3267,296.99,24,0,2.98,IN,1619288444
4,Karratha,116.8463,-20.7377,299.47,38,0,2.99,AU,1619288479
5,Makakilo City,-158.0858,21.3469,299.15,69,1,3.09,US,1619288819
6,Banyuwangi,114.3576,-8.2325,296.62,93,19,0.67,ID,1619288538
7,Aloleng,119.7824,16.1307,298.34,77,5,2.77,PH,1619288542
8,Khed,73.3833,17.7167,298.48,64,10,1.65,IN,1619288576
9,Beira,34.8389,-19.8436,296.15,83,0,3.94,MZ,1619288579


In [36]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
hotels = []
for i in range(len(filtered_city_data)):
    params = {
        "location": f"{filtered_city_data['Lat'][i]}, {filtered_city_data['Lng'][i]}",
        "radius": '5000',
        "type": 'hotel',
        "key": g_key,
    }
    response = requests.get(base_url, params=params).json()
    hotels.append(response)

In [100]:
names = []
h_lats = []
h_lngs = []
for i in range(len(filtered_city_data)):
    try:
        names.append(hotels[i]['results'][1]['name'])
        h_lats.append(hotels[i]['results'][1]['geometry']['location']['lat'])
        h_lngs.append(hotels[i]['results'][1]['geometry']['location']['lng'])
    except:
        names.append('None')
        h_lats.append('None')
        h_lngs.append('None')

filtered_city_data['Hotel_Name'] = names
filtered_city_data['Hotel_Lat'] = h_lats
filtered_city_data['Hotel_Lng'] = h_lngs

filtered_city_data = filtered_city_data[filtered_city_data['Hotel_Name'] != 'None']

In [111]:
filtered_city_data = filtered_city_data.reset_index()
del filtered_city_data['index']
filtered_city_data

In [161]:
locations = [(filtered_city_data['Lat'][i],filtered_city_data['Lng'][i]) for i in range(len(filtered_city_data))]
hover_text = [f"Name: {filtered_city_data['Hotel_Name'][i]} \nCity: {filtered_city_data['City'][i]} \nCountry: {filtered_city_data['Country'][i]}" for i in range(len(filtered_city_data))]
markers = gmaps.marker_layer(
        locations= locations,
        hover_text=hover_text
    )
m.add_layer(markers)
m

Figure(layout=FigureLayout(height='420px'))

'Name: Hotel Konford City: San Vicente de Cañete Country: PE'

[(-13.0751, -76.3835),
 (-24.7911, -50.0119),
 (-10.1667, 123.5833),
 (30.3267, 76.4003),
 (-20.7377, 116.8463),
 (21.3469, -158.0858),
 (-8.2325, 114.3576),
 (16.1307, 119.7824),
 (17.7167, 73.3833),
 (-19.8436, 34.8389),
 (11.2833, -15.8333),
 (34.8761, 33.586),
 (31.5791, -90.4407),
 (-29.6641, 22.7474),
 (37.1041, -113.5841),
 (-21.7397, -44.3092)]